In [5]:
import os
from pathlib import Path
from usv_detection import construct_csv_from_wav_file
from ava_convert_our_csv import convert_our_csv
from ava_evaluate_overlap import load_from_txt, evaluate_overlap

WAV_FILE_DIR = ''  # replace with your own path to where the .WAV recordings are you want to evaluate

def load_recordings(wav_file_dir):
    recordings = []
    for wav_file in os.listdir(wav_file_dir):
        if wav_file[-4:] == '.WAV':
            if not 'SOX' in wav_file:
                wav_file = wav_file.split('.')[0]
                recordings.append(wav_file)

    return recordings


def convert_csv_to_txt(recordings, wav_file_dir, csv_dir, output_dir):
    for recording in recordings:
        csv_path = csv_dir + recording + '.csv'
        wav_path = wav_file_dir + recording + '.WAV'
        output_path = output_dir + recording + '.txt'
        convert_our_csv(csv_path, wav_path, output_path=output_path)


def compute_metrics(recordings, min_iou_for_hit, dir_ground_truth_txts, dir_predicted_txts, dir_output):
    output_file = os.path.join(dir_output, "precision_recall_results_" + str(round(min_iou_for_hit, 1)).replace('.', '_') + ".txt")

    with open(output_file, 'w') as file:
        for recording in recordings:
            file.write('---------------------------' + recording + "----------------------------\n")
            print("------------------ " + recording + " ----------------------------")
            file_path_gt = dir_ground_truth_txts + recording + '.txt'
            start_times_gt, end_times_gt = load_from_txt(file_path_gt)

            file.write("ours\n")
            file_path_our_pred = dir_predicted_txts + recording + '.txt'
            start_times_our_pred, end_times_our_pred = load_from_txt(file_path_our_pred)

            precision, recall, true_positives, false_positives, false_negatives = evaluate_overlap(start_times_gt,
                                                                                                   end_times_gt,
                                                                                                   start_times_our_pred,
                                                                                                   end_times_our_pred,
                                                                                                   min_iou_for_hit=min_iou_for_hit)
            file.write(f"precision: {precision}\n")
            file.write(f"recall: {recall}\n")
            file.write(f"true_positives: {true_positives}\n")
            file.write(f"false_positives: {false_positives}\n")
            file.write(f"false_negatives: {false_negatives}\n")
            print("precision: {}".format(precision))
            print("recall: {}".format(recall))
            print("true_positives: {}".format(true_positives))
            print("false_positives: {}".format(false_positives))
            print("false_negatives: {}".format(false_negatives))

In [6]:
# (Optionally) Run our detection algorithm
CSV_OUTPUT_DIR_OUR_DETECTION = 'segmentation_results_our/raw/' # where to write the detection results to
Path(CSV_OUTPUT_DIR_OUR_DETECTION).mkdir(parents=True, exist_ok=True) # create folder if not exists

for wav_file in os.listdir(WAV_FILE_DIR):
    if wav_file[-4:] == '.WAV':
        if not 'SOX' in wav_file:
            print(wav_file)
            wav_file_path = os.path.join(WAV_FILE_DIR, wav_file)
            csv_file_path = CSV_OUTPUT_DIR_OUR_DETECTION + wav_file[:-4] + ".csv"
            construct_csv_from_wav_file(wav_file_path, csv_file_path)

# (Optionally) Convert our detection predictions from .csv to .txt
recordings = load_recordings(WAV_FILE_DIR)
OUTPUT_DIR_OUR_DETECTION_CONVERTED = 'data/detection_predictions/'
Path(OUTPUT_DIR_OUR_DETECTION_CONVERTED).mkdir(parents=True, exist_ok=True) # create folder if not exists
convert_csv_to_txt(recordings=recordings,
                   wav_file_dir=WAV_FILE_DIR,
                   csv_dir=CSV_OUTPUT_DIR_OUR_DETECTION,
                   output_dir=OUTPUT_DIR_OUR_DETECTION_CONVERTED)

glu397_9874_p8_4_1.WAV
detected 487 calls
r142l_144_p4_4_1.WAV
detected 133 calls
glu397_9874_p8_2_0.WAV
detected 205 calls
r142l_144_p4_5_0.WAV
detected 157 calls
r142l_133_p12_1_0.WAV
detected 53 calls
r142l_129_p12_1_0.WAV
detected 161 calls


In [9]:
# our .csv data has different formats and MUPET also has another format,
# so first get all files into the same format (.txt with two columns, one for start time one for end time)

recordings = load_recordings(WAV_FILE_DIR)
CSV_DIR_GROUND_TRUTH = 'data/labeled/'
OUTPUT_DIR_GROUND_TRUTH_CONVERTED = 'data/labeled_converted/'   # the .txt files will be saved to this folder
Path(OUTPUT_DIR_GROUND_TRUTH_CONVERTED).mkdir(parents=True, exist_ok=True) # create folder if not exists

# convert the ground truth .csv to .txt format
convert_csv_to_txt(recordings=recordings,
                   wav_file_dir=WAV_FILE_DIR,
                   csv_dir=CSV_DIR_GROUND_TRUTH,
                   output_dir=OUTPUT_DIR_GROUND_TRUTH_CONVERTED)

In [10]:
# Run the actual evaluation for computing precision, recall, true and false positives and false negatives

OUTPUT_DIR_FOR_FINAL_RESULTS = "segmentation_metric_results/"
DIR_GROUND_TRUTH_TXT = OUTPUT_DIR_GROUND_TRUTH_CONVERTED # folder where the ground truth detections (in .txt format) are
DIR_PREDICTED_TXT = OUTPUT_DIR_OUR_DETECTION_CONVERTED # the folder where the predicted detections (in .txt format) are

Path(OUTPUT_DIR_FOR_FINAL_RESULTS).mkdir(parents=True, exist_ok=True) # create folder if not exists

recordings = load_recordings(WAV_FILE_DIR)

min_iou_values_for_hit = [0.1 * i for i in range(9)]

for min_iou_for_hit in min_iou_values_for_hit:
    compute_metrics(recordings=recordings,
                    min_iou_for_hit=min_iou_for_hit,
                    dir_ground_truth_txts=DIR_GROUND_TRUTH_TXT,
                    dir_predicted_txts=DIR_PREDICTED_TXT,
                    dir_output=OUTPUT_DIR_FOR_FINAL_RESULTS)

------------------ glu397_9874_p8_4_1 ----------------------------
precision: 1.0
recall: 1.0
true_positives: 553
false_positives: 0
false_negatives: 0
------------------ r142l_144_p4_4_1 ----------------------------
precision: 1.0
recall: 1.0
true_positives: 133
false_positives: 0
false_negatives: 0
------------------ glu397_9874_p8_2_0 ----------------------------
precision: 1.0
recall: 1.0
true_positives: 246
false_positives: 0
false_negatives: 0
------------------ r142l_144_p4_5_0 ----------------------------
precision: 1.0
recall: 1.0
true_positives: 145
false_positives: 0
false_negatives: 0
------------------ r142l_133_p12_1_0 ----------------------------
precision: 1.0
recall: 1.0
true_positives: 45
false_positives: 0
false_negatives: 0
------------------ r142l_129_p12_1_0 ----------------------------
precision: 1.0
recall: 1.0
true_positives: 170
false_positives: 0
false_negatives: 0
------------------ glu397_9874_p8_4_1 ----------------------------
precision: 0.957230142566191